In [1]:
import pandas as pd
import requests
import json

from datetime import datetime

from _const import * # paths 
from _func import * # lambdas
pd.set_option('display.max_rows', 500)

In [2]:
datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")

'20210819-10:52:27'

In [3]:
queries_list = PATHS.QUERIES.ls()
print(queries_list)

[WindowsPath('queries/guardians.graphql')]


In [4]:
queries_dict = {}

for q in queries_list:
    q_name = q.parts[-1].split(".")[0]
    with open(q) as f:
        q_txt = f.read()
    queries_dict[q_name] = q_txt

In [34]:
responses_list = []

for q_name, q_statemet in queries_dict.items():
    r = requests.post(
        URLS.SUBGRAPH.COURT.MAINNET,
        json={'query': q_statemet})
    json_data = json.loads(r.text)
    len_res = len(json_data['data'][q_name])

    responses_list.append((q_name, json_data))

In [35]:
len_res

100

In [31]:
len(json_data['data']['guardians'])

100

In [6]:
df_guardians = pd.DataFrame(json_data['data']['guardians'])
df_guardians.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   activeBalance          100 non-null    object
 1   availableBalance       100 non-null    object
 2   createdAt              100 non-null    object
 3   deactivationBalance    100 non-null    object
 4   id                     100 non-null    object
 5   lockedBalance          100 non-null    object
 6   treeId                 100 non-null    object
 7   withdrawalsLockTermId  100 non-null    object
dtypes: object(8)
memory usage: 6.4+ KB


In [7]:
df_guardians.head()

,activeBalance,availableBalance,createdAt,deactivationBalance,id,lockedBalance,treeId,withdrawalsLockTermId
0,880828164821533660463,0,1628932243,0,0x01704865b3404391ea00137c4bb00855515df802,0,83,0
1,811700000000000000000,0,1628229988,0,0x03000a0ecb1898040c26cd418570cd480775f0da,0,34,0
2,33875561690053511743407,0,1628945946,0,0x06b172c63730f0ccfb40ea35e726ebf4e930c9e3,0,85,0
3,135421986952300156491,0,1628186705,0,0x098d9a770f3bd88bef60e72c833ab8db240d5928,0,25,0
4,4846740038258336017978,0,1628207854,0,0x0a3bfd31570df77c243379606ef1fca7ddd2ec74,0,29,0


In [8]:
from bq import BQ_table
table = BQ_table(BQ_guardians.dataset, BQ_guardians.table)

In [9]:
table.exists

True

In [10]:
if not table.exists:
    table.create_table_from_df(df_guardians,
    partitioned_day=True)

In [15]:
errs = table.uplaoad_df_to_bq(df_guardians)

In [16]:
errs

[]

In [17]:
df_guardians.shape

(100, 8)

In [18]:
df_guardians.head()

,activeBalance,availableBalance,createdAt,deactivationBalance,id,lockedBalance,treeId,withdrawalsLockTermId
0,880828164821533660463,0,1628932243,0,0x01704865b3404391ea00137c4bb00855515df802,0,83,0
1,811700000000000000000,0,1628229988,0,0x03000a0ecb1898040c26cd418570cd480775f0da,0,34,0
2,33875561690053511743407,0,1628945946,0,0x06b172c63730f0ccfb40ea35e726ebf4e930c9e3,0,85,0
3,135421986952300156491,0,1628186705,0,0x098d9a770f3bd88bef60e72c833ab8db240d5928,0,25,0
4,4846740038258336017978,0,1628207854,0,0x0a3bfd31570df77c243379606ef1fca7ddd2ec74,0,29,0


In [14]:
# https://cloud.google.com/bigquery/docs/nested-repeated
# https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types

In [21]:
df_guardians['createdAt'].str.len().value_counts()

10    100
Name: createdAt, dtype: int64